In [1]:
#!/usr/bin/env python3 

In [5]:
# Importing the necessary modules 
import numpy as np 
import imutils 
import pickle
import cv2 
import os 
import tensorflow as tf 
from imutils import paths 
from sklearn.svm import SVC   
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from tensorflow.keras.utils import to_categorical

In [6]:
# Setting path to the dataset and model directory 
img_dir = "dataset" 
embeddings = "models/embeddings.pickle"
detector_model = "models/face_detection_model" 
embedding_model = "models/embeddingModel.t7" 
recognize_model = "models/output/recognizer.pickle" 
label_model = "models/output/le.pickle"
X_encoder = "models/output/encoder.h5"

# Setting the confidence value 
confidence_value = 0.75

In [7]:
# load the model from disk 
proto_path = os.path.join(detector_model, "deploy.prototxt")
model_path = os.path.join(detector_model, "res10.caffemodel")

# Creating the detector 
detector = cv2.dnn.readNetFromCaffe(proto_path, model_path)

# Loading the serialized face embeddings model from disk 
embedder = cv2.dnn.readNetFromTorch(embedding_model) 

# Specifying the path to the images dataset
# Initialize our list of extracted facial embeddings that cooresponds to the 
# People names. 
image_path = list(paths.list_images(img_dir))
known_embeddings = []
known_names = []
total = 0 

In [8]:
# Creating a loop to loop through the images in the specified path. 
# load the image, resize it to have a width of 600 pixels and then grab the image dimensions. 
# Construct a blob from the image, apply opencv deep learning face detector to localize the face 
# And ensure that at least one face was found. 
for (i, image) in enumerate(image_path): 
    label = image.split("/")[-2]
    img = cv2.imread(image) 
    img = imutils.resize(img, width=600) 
    # Getting the height and width of the image 
    (h, w) = img.shape[:2]
    # Blob construction 
    image_blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, 
                                      (300, 300), (104.0, 177.0, 123.0), swapRB=False, crop=False)
     
    # Creating an instance of the detector object by calling the setInput method to 
    # take in the image_blob values 
    detector.setInput(image_blob) 
    detections = detector.forward() 
    
    # If the length for detections is greater than one, perfom the following below 
    if len(detections) > 0:
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        
        # Ensure that the detections with the larges probability also means our minimum test 
        # Thus filtering off weak detections. 
        if confidence >= confidence_value: 
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')
            
            # Extract the face Region of intreast ROI and grab the ROI dimensions 
            face = img[startX:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            
            # Ensure that the face width and height are sufficiently large. 
            if fW < 20 or fH < 20: 
                continue 
                
            # Construct a blob for the face ROI, then pass the blob through our embedding to obtain the 128-d dimensions 
            # Of the face 
            face_blob = cv2.dnn.blobFromImage(img, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, 
                                             crop=False)
            
            # Getting the embeddings 
            embedder.setInput(face_blob) 
            vector = embedder.forward() 
            
            # Adding the name of the person + corresponding face embeddngs to their
            # Respective lists 
            known_names.append(label)
            known_embeddings.append(vector.flatten())
            total += 1 

            
# Dumping the facial embeddings and its respective names to disk 
print("Serialized encodings: ", total)
data = {'embeddings': known_embeddings, 'names': known_names}

# Saving the data 
f = open(embeddings, 'wb')
f.write(pickle.dumps(data)) 
f.close() 

Serialized encodings:  1128


In [14]:
# Training the facial recognition model to produce the actual face recognition 
lb = LabelEncoder() 
y = lb.fit_transform(known_names)

# Using SVC 
model = SVC(C=1.0, kernel="linear", probability=True) 
model.fit(data['embeddings'], y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [15]:
# Writing the actual face recognition model to disk 
f = open(recognize_model, "wb")
f.write(pickle.dumps(model))
f.close() 

In [17]:
# Dumping the encoder 
f = open(label_model, "wb")
f.write(pickle.dumps(lb))
f.close() 

In [16]:
# Saving the standard scaler encoder  
# f = open(X_encoder, "wb")
# f.write(pickle.d2umps(encoder))
# f.close() 

In [9]:
# # Converting the encoded y-output features into categorical data for easy modelling 
# y_encoded = to_categorical(y, num_classes=len(lb.classes_))

# # Scaling the input features 
# encoder = StandardScaler() 
# X = np.array(known_embeddings)
# X = encoder.fit_transform(X) 



# Building the model 
# def DefineModel(input_dim): 
#     model = Sequential() 
#     model.add(Dense(32, input_dim=input_dim, activation="relu"))
#     model.add(Dense(8, activation="relu"))
#     model.add(Dense(2, activation="sigmoid"))

#     # Compiling the model 
#     model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    
#     # Returning the model 
#     return model 

# dim = X.shape[1]

# # 
# num_class = len(lb.classes_)

# # Creating the model 
# model = DefineModel(dim) 

# # Displaying the summary of the model 
# model.summary() 

In [1]:
# Training the model 
# H = model.fit(X, y_encoded, epochs = 200, batch_size=32, verbose=1)

In [2]:
# model.predict_proba(X[20].reshape(1, -1))

In [32]:
# Writing the actual face recognition model and label encoder 
# model to disk 
# model.save_weights(recognize_model)

# # Dumping the encoder 
# f = open(label_model, "wb")
# f.write(pickle.dumps(lb))
# f.close() 

In [34]:
len(lb.classes_)

2

In [35]:
dim

128

In [36]:
lb.classes_

array(['chinedu', 'unknown'], dtype='<U7')

In [37]:
X[20].reshape(1, -1).shape

(1, 128)